In [186]:
%%time
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import json
import datetime
import lxml.html as LH

from math import sqrt, exp, log, erf    # For Black Scholes

from decimal import *
getcontext().prec = 5

# Interest rate (10-year bond yield)
inturl = requests.get("https://countryeconomy.com/bonds/india").content
intrate = float(LH.fromstring(inturl).find_class('numero')[0].text)/100

# Set today's days for DTE calculation
today = datetime.datetime.now().date()

# Get NSE Scrip expiry dates
expurl = "https://www.nseindia.com/live_market/dynaContent/live_watch/fomwatchsymbol.jsp?key=NIFTY&Fut_Opt=Futures"

exphtml = requests.get(expurl).content

fnotable = pd.read_html(exphtml, match='Expiry Date', header=0)[0]
fnoexpiry = fnotable['Expiry Date'][0:1]   # Takes only the first date

# Get the list of equity scrips
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1]          # It's the second table in the url

# Remove VIX and NIFTY 
symbol = paisa.loc[~(paisa.Symbol.str.contains('VIX') | paisa.Symbol.str.contains('NIFTY'))].Symbol

# Replace & by %26 for NSE
symbol.replace('&', '%26')

# Sort the symbols
symbol = symbol.sort_values(axis=0)

# Combine Expiry and Scrips to a new url DataFrame
urldf = pd.concat([symbol] * len(fnoexpiry), keys=fnoexpiry).reset_index().drop('level_1', axis=1)

# NSE URLs
urlbase = "https://www.nseindia.com/live_market/dynaContent/live_watch/"
chainurl = urlbase + "option_chain/optionKeys.jsp?&instrument=OPTSTK&symbol="
bandurl = urlbase + "get_quote/GetQuote.jsp?symbol="
volatilityurl = urlbase + "get_quote/GetQuoteFO.jsp?instrument=FUTSTK&underlying="


# Make a Dataframe of the NSE URLs
urldf['ChainURL'] = chainurl + urldf['Symbol'] + '&date=' + urldf['Expiry Date']
urldf['BandURL'] = bandurl + urldf['Symbol']
urldf['VolURL'] = volatilityurl + urldf['Symbol']

pd.set_option('display.max_colwidth', -1)

urldf = urldf.head() # Keep only the first 5 rows for brevity

# Get the option chain function
def getchain(u, e, s, b, v):
    chainhtml = requests.get(u).content
    chain = pd.read_html(chainhtml)[1][:-1]  # read the first table and drop the total
    chain.columns=chain.columns.droplevel(0) # drop the first row of the header
    chain = chain.drop('Chart', 1)           # drop the charts
    
    # json for band
    bandhtml = requests.get(b).text  #remove IFCI when put in function!!
    bandsoup = BeautifulSoup(bandhtml, 'html.parser')
    data = bandsoup.find(id='responseDiv').text.strip()
    d1 = json.loads(data)
    
    #json for volatility
    volhtml = requests.get(v).text  #remove IFCI when put in function!!
    volsoup = BeautifulSoup(volhtml, 'html.parser')
    data = volsoup.find(id='responseDiv').text.strip()
    d2 = json.loads(data)
    
    # Dividend rate
    divurl = "https://finance.google.com/finance?q=NSE:"+s
    page = requests.get(divurl)
    root = LH.fromstring(page.content)
    try:
        dividend = float(root.findall('.//table')[2].text_content().strip().split("\n")[2].split('/')[0])/100
    except Exception:
        dividend = 0
    
    try:                                     # match='Underlying Stock:' gives errors when table is not found
        underlyingtbl = pd.read_html(chainhtml, match='Underlying Stock:')[0][1]
        underlying = underlyingtbl.iloc[0]
        u = float(underlying.split(' ')[3])
        
        # parse for Lo52, Hi52, LoBand, HiBand, MarginRate, WhenLo52, WhenHi52
        Lo52 = float(d1['data'][0]['low52'])
        Hi52 = float(d1['data'][0]['high52'])
        WhenLo52 = d1['data'][0]['cm_adj_low_dt']
        WhenHi52 = d1['data'][0]['cm_adj_high_dt']
        LoBand = float(d1['data'][0]['pricebandlower'])
        HiBand = float(d1['data'][0]['pricebandupper'])
        MarginRate = float(d1['data'][0]['applicableMargin'])/100
        
        
        # get the Days-To-Expiry (DTE)
        exp = datetime.datetime.strptime(e, '%d%b%Y').date()
        DTE = (exp - today)
        
        # parse for daily volatility
        DailyVol = float(d2['data'][0]['dailyVolatility'])/100

        Volatility = float(d2['data'][0]['annualisedVolatility'])/100        
        # Volatility = DailyVol * sqrt(DTE.days)    #Volatility of remaining days-to-expiry
    

        chain['Underlying'] = u              # price of the underlying
        chain['Expiry'] = e                  # expiry date
        chain['Symbol'] = s                  # symbol
        chain['Lo52'] = Lo52
        chain['Hi52'] = Hi52
        chain['WhenLo52'] = WhenLo52
        chain['WhenHi52'] = WhenHi52
        chain['LoBand'] = LoBand
        chain['HiBand'] = HiBand
        chain['MarginRate'] = MarginRate
        chain['DailyVol']= DailyVol
        chain['DTE'] = DTE.days
        chain['Volatility'] = Volatility
        chain['Dividend'] = dividend

        return chain
    
    except Exception:                        # pass the exception (to store exceptions later)
        pass

# vectorize getchain function, pass arguments of urldf to it and rename output columns
v = np.vectorize(getchain)            
output = pd.concat(v(urldf.ChainURL, urldf['Expiry Date'], urldf.Symbol, urldf.BandURL, urldf.VolURL)).reset_index(drop=True)

output['IntRate'] = intrate

#Black & Scholes function
def calc_bs(undprice, strike, time, rate, sigma, divrate):

    #statistics
    sigTsquared = sqrt(Decimal(time)/365)*sigma
    edivT = exp((-divrate*time)/365)
    ert = exp((-rate*time)/365)
    d1 = (log(undprice*edivT/strike)+(rate+.5*(sigma**2))*time/365)/sigTsquared
    d2 = d1-sigTsquared
    Nd1 = (1+erf(d1/sqrt(2)))/2
    Nd2 = (1+erf(d2/sqrt(2)))/2
    iNd1 = (1+erf(-d1/sqrt(2)))/2
    iNd2 = (1+erf(-d2/sqrt(2)))/2

    #outputs
    callPrice = round(undprice*edivT*Nd1-strike*ert*Nd2, 2)
    putPrice = round(strike*ert*iNd2-undprice*edivT*iNd1, 2)
    prob = iNd2
    
    return callPrice, putPrice, prob

# Vectorize Black Scholes and get BSCall and BSPut
vbs = np.vectorize(calc_bs)
df =vbs(output.Underlying, output['Strike Price'], output.DTE.astype(float), output.IntRate, output.Volatility, output.Dividend)
df = pd.DataFrame(np.column_stack(df), columns = ['BSCall', 'BSPut', 'Probability'])
output = output.join(df)

# Rename columns to meaningful
chainheader = ['Call_OI', 'Call_OI_Change', 'Call_Volume', 'Call_IV', \
               'Call_LTP', 'Call_Net_Change', 'Call_BidQty', 'Call_BidPrice', 'Call_AskPrice', 'Call_AskQty', \
               'Strike', 'Put_BidQty', 'Put_BidPrice', 'Put_AskPrice', 'Put_AskQty', 'Put_Net_Change', \
               'Put_LTP', 'Put_IV', 'Put_Volume', 'Put_OI_Change', 'Put_OI', \
               'Underlying', 'Expiry', 'Symbol', 'Lo52', 'Hi52', 'WhenLo52', 'WhenHi52', 'LoBand', 'HiBand', \
               'MarginRate', 'DailyVol', 'DTE', 'Volatility', 'Dividend', 'IntRate', 'BSCall', 'BSPut', 'Probability']

output.columns = chainheader

#Weed out values with no Last Traded Price (LTP)
output = output.replace("-",0)              # Make '-' to 0)
output = output[(output.Call_LTP != 0) | (output.Put_LTP != 0)]

#Sort by Symbol and Strike - to show highs in the top and lows in the bottom (similar to IBKR)
output = output.sort_values(by=['Symbol', 'Strike'], ascending=[1,0]).reset_index(drop=True)

Wall time: 32.1 s


In [182]:
output[['Symbol', 'Strike', 'Underlying', 'Call_LTP', 'BSCall', 'MarginRate', 'Probability']]


,Symbol,Strike,Underlying,Call_LTP,BSCall,MarginRate,Probability
0,ADANIENT,235.0,199.1,1.05,0.24,0.1474,9.709663e-01
1,ADANIENT,230.0,199.1,1.40,0.43,0.1474,9.507272e-01
2,ADANIENT,225.0,199.1,2.15,0.75,0.1474,9.197447e-01
3,ADANIENT,220.0,199.1,2.90,1.26,0.1474,8.747592e-01
4,ADANIENT,215.0,199.1,3.70,2.03,0.1474,8.130416e-01
5,ADANIENT,210.0,199.1,5.00,3.15,0.1474,7.333651e-01
6,ADANIENT,205.0,199.1,6.70,4.71,0.1474,6.370071e-01
7,ADANIENT,200.0,199.1,8.75,6.79,0.1474,5.283760e-01
8,ADANIENT,195.0,199.1,11.00,9.43,0.1474,4.148192e-01
9,ADANIENT,190.0,199.1,14.75,12.62,0.1474,3.053911e-01


In [ ]:
# datetime.datetime.strptime(str(datetime.datetime.now()), '%Y-%m-%d')
dt = str(datetime.datetime.now())
filename = datetime.datetime.strptime(dt, '%Y-%m-%d_%I-%M')
filename

In [184]:
paisa

,Symbol,Mlot,SpMgn%,ExpMgn%,TotMgn%,SpMgnPerShare,ExpMgnPerShr,TotMgnPerShr,SpMgnPerLt,ExpMgnPerLt,TotMgnPerLt
0,RCOM,28000,37.20,9.46,46.66,12.2400,3.1123,15.3523,342720.00,87145.5200,429865.5200
1,RNAVAL,9000,24.21,5.00,29.21,13.9500,2.8800,16.8300,125550.00,25920.0000,151470.0000
2,JPASSOCIAT,34000,17.49,7.50,24.99,4.1200,1.7663,5.8863,140080.00,60052.5000,200132.5000
3,RPOWER,13000,19.37,5.00,24.37,11.1400,2.8750,14.0150,144820.00,37375.0000,182195.0000
4,INDIAVIX,900,16.89,5.00,21.89,233.8420,69.2250,303.1050,210457.80,62302.5000,272794.5000
5,INDIAVIX,900,16.81,5.00,21.81,233.8271,69.5500,303.4300,210444.39,62595.0000,273087.0000
6,INDIAVIX,900,16.81,5.00,21.81,233.8800,69.5500,303.4300,210492.00,62595.0000,273087.0000
7,IFCI,22000,15.32,5.00,20.32,4.6200,1.5075,6.1275,101640.00,33165.0000,134805.0000
8,JINDALSTEL,4500,14.24,5.00,19.24,35.5400,12.4750,48.0150,159930.00,56137.5000,216067.5000
9,GMRINFRA,45000,13.25,5.00,18.25,2.8100,1.0600,3.8700,126450.00,47700.0000,174150.0000
